In [1]:
#COPY OF LOAD TO TEST NEW REFACTORING

In [ ]:
#Note: this code is written as an assignment in a Helsinki Uni course on Deep Learning and is heavily influenced by
#starter code provided by the lecturers Hande Celikkanat and Roman Yangarber

import os
import pandas as pd
import numpy as np

In [2]:
#DATA SHOULD BE IN "data/" folder NOW
#HERES HOW TO DOWNLOAD DATA DIRECTLY (should skip if already has data):
from src import data_download
data_download.fetch_data()

Folder already has folders 'annotations' and 'images'.
Assuming you already have the data and skipping fetch.


In [3]:
import torch
import torch.optim as optim
import torch.utils.data
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F

In [4]:
from PIL import Image
from torchvision.io import read_image
from torch.utils.data import Dataset

In [5]:
DATA_DIR = 'data/images'

In [6]:
train_transform = transforms.Compose([transforms.Grayscale()])
test_transform = transforms.Compose([transforms.Grayscale()])

In [10]:
#this code is an edited version of the code found at https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

class CustomImageDataset(Dataset):
    def __init__(self, annotations_df, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_df
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, "im"+str(idx+1)+".jpg")
        image = Image.open(img_path)
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Grayscale()
            #transforms.RandomResizedCrop(256)
        ])
        img_tensor = transform(image)
        label = torch.from_numpy(np.array(self.img_labels.iloc[idx])).float()#NOTE! This should be fixed (the float issue)
        if self.transform:
            img_tensor = self.transform(img_tensor)
        if self.target_transform:
            label = self.target_transform(label)
        return img_tensor, label


In [8]:
#NEW WAY TO LOAD IN DATA:
from src import data_handling
train, test, val = data_handling.get_target_dfs(train=0.6, test=0.2, val=0.2)

In [11]:
train_loader = torch.utils.data.DataLoader(dataset=CustomImageDataset(train, DATA_DIR), batch_size=50, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=CustomImageDataset(test, DATA_DIR), batch_size=50, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=CustomImageDataset(val, DATA_DIR), batch_size=50, shuffle=True)

In [12]:
NUM_CLASSES=14

In [13]:
class CNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(CNN, self).__init__()
        self.dropout = nn.Dropout(0.05)
        self.features = nn.Sequential(
          nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.BatchNorm2d(16),
          nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.BatchNorm2d(32)
        )

        self.classify = nn.Sequential(
          nn.Linear(32 * 32 * 32, 60),
          nn.Linear(60, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        #print(x.shape)
        x = x.view(-1, 32 * 32 * 32)
        x = self.classify(x)
        return x

In [14]:
model = CNN(NUM_CLASSES)

In [15]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [16]:
model = CNN().to(device)

In [17]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.01)

In [ ]:
for epoch in range(5):
    train_loss = 0
    train_correct = 0
    total = 0
    for batch_num, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        result = model.forward(data)
        probs = torch.softmax(result, dim=1)
        #print(probs)
        #winners = probs.argmax(dim=1)
        #print(winners)
        optimizer.zero_grad()
        loss = loss_function(result, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        print(train_loss / (batch_num+1))
        print('Training: Epoch %d - Batch %d/%d: Loss: %.4f ' % 
              (epoch, batch_num, len(train_loader), train_loss / (batch_num + 1)))
        #train_correct += (winners == target).sum().item()
        #total = total + BATCH_SIZE_TRAIN
        #print('Training: Epoch %d - Batch %d/%d: Loss: %.4f | Train Acc: %.3f%% (%d/%d)' % 
        #      (epoch, batch_num, len(train_loader), train_loss / (batch_num + 1), 
        #       100. * train_correct / total, train_correct, total))